## 256x256 diffusion model presented in Guided diffusion official github repo is not compatible with their unet implementation.
## However Repaint modified it to make it compatible.

In [1]:

import os
import argparse
import torch as th
import torch.nn.functional as F
import time
import conf_mgt
from utils import yamlread
from guided_diffusion import dist_util
from tqdm import tqdm
# Workaround
try:
    import ctypes
    libgcc_s = ctypes.CDLL('libgcc_s.so.1')
except:
    pass


from guided_diffusion.script_util import (
    NUM_CLASSES,
    model_and_diffusion_defaults,
    classifier_defaults,
    create_model_and_diffusion,
    create_classifier,
    select_args,
)  # noqa: E402

from guided_diffusion.script_util import (
    sr_model_and_diffusion_defaults,
    sr_create_model_and_diffusion,
)
def toU8(sample):
    if sample is None:
        return sample

    sample = ((sample + 1) * 127.5).clamp(0, 255).to(th.uint8)
    sample = sample.permute(0, 2, 3, 1)
    sample = sample.contiguous()
    sample = sample.detach().cpu().numpy()
    return sample

import torch
import torchvision.transforms as T

In [ ]:
**select_a

{'paper_face_mask': {'mask_loader': True,
  'gt_path': './data/datasets/gts/face',
  'mask_path': './data/datasets/gt_keep_masks/face',
  'image_size': 256,
  'class_cond': False,
  'deterministic': True,
  'random_crop': False,
  'random_flip': False,
  'return_dict': True,
  'drop_last': False,
  'batch_size': 1,
  'return_dataloader': True,
  'offset': 0,
  'max_len': 8,
  'paths': {'srs': './log/face_example/inpainted',
   'lrs': './log/face_example/gt_masked',
   'gts': './log/face_example/gt',
   'gt_keep_masks': './log/face_example/gt_keep_mask'}}}

In [2]:
conf = conf_mgt.conf_base.Default_Conf()
conf.update(yamlread('confs/my_conf-2.yml'))
print(conf.inpa_inj_sched_prev) #Are you about to use RePainting ?print(conf.diffusion_steps)
print(conf.intermediate_steps)

True
True


In [3]:
model_and_diffusion_defaults()

{'image_size': 64,
 'num_channels': 128,
 'num_res_blocks': 2,
 'num_heads': 4,
 'num_heads_upsample': -1,
 'num_head_channels': -1,
 'attention_resolutions': '16,8',
 'channel_mult': '',
 'dropout': 0.0,
 'class_cond': False,
 'use_checkpoint': False,
 'use_scale_shift_norm': True,
 'resblock_updown': False,
 'use_fp16': False,
 'use_new_attention_order': False,
 'learn_sigma': False,
 'diffusion_steps': 1000,
 'noise_schedule': 'linear',
 'timestep_respacing': '',
 'use_kl': False,
 'predict_xstart': False,
 'rescale_timesteps': False,
 'rescale_learned_sigmas': False}

In [3]:
print("Start", conf['name'])

device = dist_util.dev(conf.get('device'))


model, diffusion = create_model_and_diffusion(
    **select_args(conf, model_and_diffusion_defaults().keys()), conf=conf
)


Start my_conf


In [4]:
model.load_state_dict(
    dist_util.load_state_dict(os.path.expanduser(
        conf.model_path), map_location="cpu")
)
model.to(device)
if conf.use_fp16:
    model.convert_to_fp16()
model.eval()

show_progress = conf.show_progress

import numpy as np
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('Nombre de paramètres : ', params)

Nombre de paramètres :  552814086


In [ ]:
#GENERATE A SAMPLE AND SAVE IT (without repaint)
n_samples = 1
model_kwargs = {}
sample_fn = (
    diffusion.p_sample_loop if not conf.use_ddim else diffusion.ddim_sample_loop
)
intermediates,finals = sample_fn(
    model,
    (n_samples, 3, 256, 256), #REMPLACER PAR CONF.IMAGE_SIZE
    clip_denoised=conf.clip_denoised,
    model_kwargs=model_kwargs,
    device=device,
    progress=show_progress,
    return_all=True,
    conf=conf
)

In [6]:
#save evey final sample
transform = T.ToPILImage()

f_samples = finals['sample']
f_samples = ((f_samples + 1) * 127.5).clamp(0, 255).to(th.uint8)
f_samples = f_samples.contiguous()

i=0
for sample in f_samples:
    img = transform(sample)
    img.save("samples/sample_"+str(i)+".jpeg")
    i+=1

In [10]:
#GENERATE A SAMPLE AND SAVE IT (without repaint)
for i in range(50):
    n_samples = 1
    model_kwargs = {}
    sample_fn = (
        diffusion.p_sample_loop if not conf.use_ddim else diffusion.ddim_sample_loop
    )
    intermediates,finals = sample_fn(
        model,
        (n_samples, 3, 256, 256), #REMPLACER PAR CONF.IMAGE_SIZE
        clip_denoised=conf.clip_denoised,
        model_kwargs=model_kwargs,
        device=device,
        progress=show_progress,
        return_all=True,
        conf=conf
    )

    transform = T.ToPILImage()

    f_samples = finals['sample']
    f_samples = ((f_samples + 1) * 127.5).clamp(0, 255).to(th.uint8)
    f_samples = f_samples.contiguous()

    for sample in f_samples:
        img = transform(sample)
        img.save("samples/sample_"+str(i)+".jpeg")


  0%|          | 0/4570 [00:00<?, ?it/s]

  0%|          | 0/4570 [00:00<?, ?it/s]

  0%|          | 0/4570 [00:00<?, ?it/s]

  0%|          | 0/4570 [00:00<?, ?it/s]

  0%|          | 0/4570 [00:00<?, ?it/s]

...

  0%|          | 0/4570 [00:00<?, ?it/s]

  0%|          | 0/4570 [00:00<?, ?it/s]

  0%|          | 0/4570 [00:00<?, ?it/s]

  0%|          | 0/4570 [00:00<?, ?it/s]

  0%|          | 0/4570 [00:00<?, ?it/s]

In [22]:
#generate gif



samples = torch.stack(intermediates)
samples = ((samples + 1) * 127.5).clamp(0, 255).to(th.uint8)
samples = samples.permute(1,0, 2, 3, 4)
samples = samples.contiguous()


step = 10
i=0
for sample_intermediates in list(samples):
    
    imgs = (transform(sample_intermediates[int(j*step)]) for j in range(int(len(sample_intermediates)/step)))
    img = next(imgs)  # extract first image from iterator
    img.save(fp='samples/gif_'+str(i)+'.gif', format='GIF', append_images=imgs,
             save_all=True, duration=50, loop=0)
    i+=1
    print(i)

1
2
3
4
5


In [ ]:
# https://colab.research.google.com/drive/12a_Wrfi2_gwwAuN3VvMTwVMz9TfqctNj#scrollTo=7zAqFEykBHDL TO DO

In [2]:
#Upscaling dont work for now


conf_us = conf_mgt.conf_base.Default_Conf()
conf_us.update(yamlread('confs/upsample_conf.yml'))

def args_to_dict(args, keys):
    return {k: getattr(args, k) for k in keys}
args_to_dict(conf_us, sr_model_and_diffusion_defaults().keys())

print("Start", conf_us['name'])

device = dist_util.dev(conf_us.get('device'))


model_us, diffusion_us = sr_create_model_and_diffusion(
    **args_to_dict(conf_us, sr_model_and_diffusion_defaults().keys()),conf=conf_us
)
model_us.load_state_dict(
    dist_util.load_state_dict(os.path.expanduser(
        conf_us.model_path), map_location="cpu")
)
model_us.to(device)
if conf_us.use_fp16:
    model_us.convert_to_fp16()
model_us.eval()

show_progress = conf_us.show_progress

import numpy as np
model_parameters = filter(lambda p: p.requires_grad, model_us.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('Nombre de paramètres : ', params)

Start upsample
Nombre de paramètres :  279792390


In [3]:
n_samples = 1
import torch
model_kwargs = {"y":torch.randn(1,3,64,64)}


intermediates,finals = diffusion_us.p_sample_loop(
    model_us,
    (n_samples, 3, 64, 64),
    clip_denoised=conf_us.clip_denoised,
    model_kwargs=model_kwargs,
    device=device,
    progress=show_progress,
    return_all=True,
    conf=conf_us
)

  0%|          | 0/1000 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'dim'

In [3]:
noise = torch.randn(1,3,256,256)
low_res = torch.randn(1,3,64,64)
model_us(noise,torch.tensor100,low_res)

AttributeError: 'int' object has no attribute 'device'

In [34]:
torch.tensor([4]).dim

<function Tensor.dim>

In [26]:
diffusion_us.p_sample_loop(model_us,(n_samples, 3, 64, 64),return_all=True,conf=conf_us)

In [5]:
#with REPAINT
if conf.classifier_scale is not None:
    if conf.classifier_scale > 0 and conf.classifier_path:
        print("loading classifier...")
        classifier = create_classifier(
            **select_args(conf, classifier_defaults().keys()))
        classifier.load_state_dict(
            dist_util.load_state_dict(os.path.expanduser(
                conf.classifier_path), map_location="cpu")
        )

        classifier.to(device)
        if conf.classifier_use_fp16:
            classifier.convert_to_fp16()
        classifier.eval()

        def cond_fn(x, t, y=None, gt=None, **kwargs):
            assert y is not None
            with th.enable_grad():
                x_in = x.detach().requires_grad_(True)
                logits = classifier(x_in, t)
                log_probs = F.log_softmax(logits, dim=-1)
                selected = log_probs[range(len(logits)), y.view(-1)]
                return th.autograd.grad(selected.sum(), x_in)[0] * conf.classifier_scale
else:
    cond_fn = None

def model_fn(x, t, y=None, gt=None, **kwargs):
    assert y is not None
    return model(x, t, y if conf.class_cond else None, gt=gt)


print("sampling complete")

sampling complete


In [11]:
dl

In [6]:

print("sampling...")
all_images = []

dset = 'eval'

eval_name = conf.get_default_eval_name()

dl = conf.get_dataloader(dset=dset, dsName=eval_name)

for batch in tqdm(iter(dl)):

    for k in batch.keys():
        if isinstance(batch[k], th.Tensor):
            batch[k] = batch[k].to(device)

    model_kwargs = {}

    model_kwargs["gt"] = batch['GT']

    gt_keep_mask = batch.get('gt_keep_mask')
    if gt_keep_mask is not None:
        model_kwargs['gt_keep_mask'] = gt_keep_mask

    batch_size = model_kwargs["gt"].shape[0]

    if conf.cond_y is not None:
        classes = th.ones(batch_size, dtype=th.long, device=device)
        model_kwargs["y"] = classes * conf.cond_y
    else:
        classes = th.randint(
            low=0, high=NUM_CLASSES, size=(batch_size,), device=device
        )
        model_kwargs["y"] = classes

    sample_fn = (
        diffusion.p_sample_loop if not conf.use_ddim else diffusion.ddim_sample_loop
    )

    
    result = sample_fn(
        model_fn,
        (batch_size, 3, conf.image_size, conf.image_size),
        clip_denoised=conf.clip_denoised,
        model_kwargs=model_kwargs,
        cond_fn=cond_fn,
        device=device,
        progress=show_progress,
        return_all=True,
        conf=conf
    )
    
    if conf.name=='my_conf':
        result = result[1] #0 is all tuples to generate gifs
    srs = toU8(result['sample'])
    gts = toU8(result['gt'])
    lrs = toU8(result.get('gt') * model_kwargs.get('gt_keep_mask') + (-1) *
               th.ones_like(result.get('gt')) * (1 - model_kwargs.get('gt_keep_mask')))

    gt_keep_masks = toU8((model_kwargs.get('gt_keep_mask') * 2 - 1))

    conf.eval_imswrite(
        srs=srs, gts=gts, lrs=lrs, gt_keep_masks=gt_keep_masks,
        img_names=batch['GT_name'], dset=dset, name=eval_name, verify_same=False)

sampling...


  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

  0%|          | 0/4570 [00:00<?, ?it/s]

 12%|██████████▍                                                                        | 1/8 [06:19<44:15, 379.32s/it]

  0%|          | 0/4570 [00:00<?, ?it/s]

 12%|██████████▍                                                                        | 1/8 [07:35<53:06, 455.29s/it]


KeyboardInterrupt: 

In [103]:
img = batch['GT'].to(device)
mask = batch['gt_keep_mask'].to(device)

#img = torch.stack([img,img])
#mask = torch.stack([mask,mask])

model_kswargs = {}
model_kwargs["gt"] = img
model_kwargs['y'] = th.randint(
        low=0, high=NUM_CLASSES, size=(batch_size,), device=device
    )
model_kwargs['gt_keep_mask'] = mask

batch_size = model_kwargs['gt'].shape[0]

sample_fn = (
    diffusion.p_sample_loop if not conf.use_ddim else diffusion.ddim_sample_loop
)

In [ ]:
model

In [104]:

result = sample_fn(
    model_fn,
    (2, 3, conf.image_size, conf.image_size),
    clip_denoised=conf.clip_denoised,
    model_kwargs=model_kwargs,
    cond_fn=cond_fn,
    device=device,
    progress=show_progress,
    return_all=True,
    conf=conf
)


  0%|          | 0/4570 [00:00<?, ?it/s]

In [111]:
transform = T.ToPILImage()

finals = result[1]
f_samples = finals['sample']
f_samples = ((f_samples + 1) * 127.5).clamp(0, 255).to(th.uint8)
f_samples = f_samples.contiguous()
i=0
for sample in f_samples:
    img = transform(sample)
    img.save("images/sample_"+str(i)+".jpeg")
    i+=1

In [108]:

for sample in finals['sample']:

torch.Size([2, 3, 256, 256])

In [17]:





if conf.name=='my_conf':
    result = result[1] #0 is all tuples to generate gifs
srs = toU8(result['sample'])
gts = toU8(result['gt'])
lrs = toU8(result.get('gt') * model_kwargs.get('gt_keep_mask') + (-1) *
           th.ones_like(result.get('gt')) * (1 - model_kwargs.get('gt_keep_mask')))

gt_keep_masks = toU8((model_kwargs.get('gt_keep_mask') * 2 - 1))

conf.eval_imswrite(
    srs=srs, gts=gts, lrs=lrs, gt_keep_masks=gt_keep_masks,
    img_names=batch['GT_name'], dset=dset, name=eval_name, verify_same=False)

  0%|          | 0/4570 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
    srs = toU8(result[1]['sample'])
    gts = toU8(result[1]['gt'])
    lrs = toU8(result[1].get('gt') * model_kwargs.get('gt_keep_mask') + (-1) *
               th.ones_like(result[1].get('gt')) * (1 - model_kwargs.get('gt_keep_mask')))

    gt_keep_masks = toU8((model_kwargs.get('gt_keep_mask') * 2 - 1))

    conf.eval_imswrite(
        srs=srs, gts=gts, lrs=lrs, gt_keep_masks=gt_keep_masks,
        img_names=batch['GT_name'], dset=dset, name=eval_name, verify_same=False)

In [24]:
lrs

array([[[[143, 144, 156],
         [140, 142, 154],
         [144, 145, 157],
         ...,
         [104, 120, 130],
         [105, 121, 132],
         [104, 120, 131]],

        [[142, 143, 155],
         [141, 143, 155],
         [142, 143, 155],
         ...,
         [104, 120, 130],
         [102, 118, 129],
         [103, 119, 130]],

        [[142, 144, 156],
         [142, 143, 155],
         [142, 144, 156],
         ...,
         [101, 118, 129],
         [100, 118, 130],
         [103, 120, 131]],

        ...,

        [[153, 127, 116],
         [169, 134, 117],
         [168, 133, 114],
         ...,
         [ 37,  80, 104],
         [ 33,  82, 107],
         [ 31,  84, 111]],

        [[168, 130, 116],
         [173, 132, 117],
         [171, 132, 117],
         ...,
         [ 36,  81, 104],
         [ 33,  83, 107],
         [ 32,  86, 112]],

        [[169, 133, 113],
         [169, 133, 115],
         [167, 128, 113],
         ...,
         [ 35,  82, 104],
        